In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import BDay
from tqdm.notebook import tqdm # 使用 notebook 友好的进度条
import os
import warnings
warnings.filterwarnings('ignore')
# 设置全局中文字体
# 'PingFang HK' 在 macOS 上表现稳定且美观
plt.rcParams['font.sans-serif'] = ['PingFang HK', 'STHeiti', 'Arial Unicode MS'] 

# 解决保存图像时负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False 

In [37]:
fs_df = pd.read_csv('data/ind_financial_indicators_2015q1_2025q2.csv')

df = fs_df[['ts_code','ann_date','end_date','q_sales_yoy','q_profit_yoy','l3_name']].copy()

In [38]:
ind_list = [
    "制冷空调设备",    # 对应: 液冷
    "其他专用设备",    # 对应: 液冷
    "光学元件",        # 对应: 光芯片
    "通信网络设备及器件",  # 对应: 光芯片
    "半导体材料",      # 对应: 光芯片
    "锂",              # 对应: 锂矿
    "锂电池",          # 对应: 动力电池, 储能
    "电池化学品",      # 对应: 电池正极、负极、电解液
    "膜材料",          # 对应: 隔膜
    "风电整机",        # 对应: 风电风机
    "风电零部件",      # 对应: 风电轴承, 风电电缆
    "线缆部件及其他",  # 对应: 风电电缆
    "硅料硅片",        # 对应: 光伏硅料、硅片
    "光伏电池组件",    # 对应: 光伏电池
    "玻璃制造",        # 对应: 光伏玻璃
    "光伏辅材",        # 对应: 光伏玻璃
    "逆变器",          # 对应: 逆变器, 储能
    "蓄电池及其他电池" # 对应: 储能
]

In [39]:
# 在df中筛选l3_name在ind_list中的数据
df_ind = df[df['l3_name'].isin(ind_list)].copy()

In [40]:
df_ind.isnull().sum()

ts_code           0
ann_date          0
end_date          0
q_sales_yoy     781
q_profit_yoy    782
l3_name           0
dtype: int64

In [41]:
df_ind.describe()

,q_sales_yoy,q_profit_yoy
count,11002.000000,11001.000000
mean,25.994965,41.700027
std,132.270314,3374.138558
min,-570.320200,-81699.420500
25%,-7.610725,-46.976700
50%,11.431550,8.048500
75%,36.351575,76.874200
max,9502.738700,301125.465500


In [42]:
# 填充缺失值：按“报告期”和“行业”分组，然后用该组的“中位数”来填充组内的缺失值
for col in ['q_sales_yoy', 'q_profit_yoy']:
    df_ind[col] = df_ind.groupby(['end_date', 'l3_name'])[col].transform(
        lambda x: x.fillna(x.median())
    )

# 使用“市场中位数”进行截面填充,来填充第一步未填充的缺失值  
for col in ['q_sales_yoy', 'q_profit_yoy']:
    df_ind[col] = df_ind.groupby('end_date')[col].transform(
        lambda x: x.fillna(x.median())
    )
    
# 还有缺失值的行删除
df_ind.dropna(subset=['q_sales_yoy', 'q_profit_yoy'], inplace=True)

In [43]:
# 缩尾处理(将所有高于 99% 分位数的值“拉回”到 99% 分位数的值，将所有低于 1% 分位数的值“拉升”到 1% 分位数的值)
cols_to_process = ['q_sales_yoy', 'q_profit_yoy']

# 设置缩尾的百分位
p_low = 0.01  # 1%
p_high = 0.99 # 99%

for col in cols_to_process:
    # 计算1%和99%的分位数
    low_val = df_ind[col].quantile(p_low)
    high_val = df_ind[col].quantile(p_high)
    
    print(f"处理 {col}: 1% 分位数 = {low_val:.2f}, 99% 分位数 = {high_val:.2f}")
    
    # 使用 .clip() 函数进行缩尾
    df_ind[col] = df_ind[col].clip(lower=low_val, upper=high_val)

print("\n缩尾处理完成。")

处理 q_sales_yoy: 1% 分位数 = -69.37, 99% 分位数 = 281.74
处理 q_profit_yoy: 1% 分位数 = -1708.62, 99% 分位数 = 1998.76

缩尾处理完成。


In [44]:
df_ind.describe()

,q_sales_yoy,q_profit_yoy
count,11777.000000,11777.000000
mean,21.594743,35.318326
std,53.325150,365.873732
min,-69.373052,-1708.621616
25%,-6.234300,-42.808700
50%,11.845100,8.913900
75%,35.342200,72.515500
max,281.737844,1998.757308


In [45]:
df_ind['l3_name'].unique()

array(['电池化学品', '玻璃制造', '锂电池', '通信网络设备及器件', '光伏辅材', '锂', '制冷空调设备',
       '线缆部件及其他', '其他专用设备', '膜材料', '光学元件', '光伏电池组件', '半导体材料', '硅料硅片',
       '风电整机', '风电零部件', '蓄电池及其他电池', '逆变器'], dtype=object)

In [46]:
df_ind

,ts_code,ann_date,end_date,q_sales_yoy,q_profit_yoy,l3_name
236,000009.SZ,2015-04-29,2015-03-31,15.1639,-13.6133,电池化学品
237,000009.SZ,2015-08-28,2015-06-30,6.0100,-36.9593,电池化学品
238,000009.SZ,2015-10-31,2015-09-30,34.0505,703.5780,电池化学品
239,000009.SZ,2016-04-29,2016-03-31,52.7519,1.8531,电池化学品
240,000009.SZ,2016-08-30,2016-06-30,29.6020,-24.0442,电池化学品
...,...,...,...,...,...,...
163987,920682.BJ,2024-08-24,2024-06-30,22.4453,0.3290,线缆部件及其他
163988,920682.BJ,2024-10-30,2024-09-30,58.3800,51.3643,线缆部件及其他
163989,920682.BJ,2025-03-29,2024-12-31,4.4360,0.1728,线缆部件及其他
163990,920682.BJ,2025-04-29,2025-03-31,5.6347,-24.1134,线缆部件及其他


In [47]:
price_df = pd.read_csv('data/stk_ind_data_20150101_20251017.csv')
price_df = price_df[['ts_code','trade_date','close','total_mv','l3_name']]
price_df = price_df[price_df['l3_name'].isin(ind_list)]
price_df.reset_index(inplace=True,drop=True)
price_df['trade_date'] = pd.to_datetime(price_df['trade_date'], format='%Y%m%d')

In [48]:
price_df

,ts_code,trade_date,close,total_mv,l3_name
0,300151.SZ,2025-10-17,12.95,6.895989e+05,其他专用设备
1,601615.SH,2025-10-17,15.30,3.475390e+06,风电整机
2,002882.SZ,2025-10-17,28.88,1.250215e+06,线缆部件及其他
3,002506.SZ,2025-10-17,2.47,1.445028e+06,光伏电池组件
4,300035.SZ,2025-10-17,22.49,1.541525e+06,电池化学品
...,...,...,...,...,...
748672,002613.SZ,2015-01-05,7.99,3.839994e+05,玻璃制造
748673,600072.SH,2015-01-05,11.44,5.473234e+05,风电零部件
748674,300382.SZ,2015-01-05,58.58,3.118623e+05,其他专用设备
748675,002595.SZ,2015-01-05,24.59,9.836000e+05,其他专用设备


In [49]:
def robust_merge_asof(left_df, right_df, left_on, right_on, by):
    """执行稳健的合并操作，确保在合并时考虑到时间序列的顺序。"""
    print(f"--- Performing robust merge_asof on '{by}' ---")

    # 优化: 确保合并键也是 category
    #if pd.api.types.is_categorical_dtype(left_df[by]) and not pd.api.types.is_categorical_dtype(right_df[by]):
        #right_df[by] = right_df[by].astype('category')
    
    left_df = left_df.reset_index(drop=True).sort_values(by=[by, left_on])
    right_df = right_df.reset_index(drop=True).sort_values(by=[by, right_on])
    all_keys = left_df[by].unique()
    merged_chunks = []
    for key in all_keys:
        left_chunk = left_df[left_df[by] == key]
        right_chunk = right_df[right_df[by] == key]
        merged_chunk = pd.merge_asof(
            left=left_chunk, right=right_chunk, left_on=left_on,
            right_on=right_on, by=by, direction='backward'
        )
        merged_chunks.append(merged_chunk)
    print("Robust merge successful.")
    return pd.concat(merged_chunks, ignore_index=True)


In [50]:
df_ind['ann_date'] = pd.to_datetime(df_ind['ann_date'])
financial_cols = ['ts_code', 'ann_date', 'end_date', 'q_sales_yoy', 'q_profit_yoy', 'l3_name']
df_ind_subset = df_ind[financial_cols].copy()

# 【修复】使用你的新函数
merged_df = robust_merge_asof(
    price_df,                       
    df_ind_subset,                  
    left_on='trade_date',           
    right_on='ann_date',          
    by='ts_code',
)

--- Performing robust merge_asof on 'ts_code' ---
Robust merge successful.


In [51]:
# 确保只删除 _y 后缀的列 (如果存在)
if 'l3_name_y' in merged_df.columns:
    merged_df = merged_df.drop(columns=['l3_name_y'])

In [52]:
df = merged_df.dropna().copy()

In [53]:
df.rename(columns={'l3_name_x': 'l3_name'}, inplace=True)

In [54]:
df

,ts_code,trade_date,close,total_mv,l3_name,ann_date,end_date,q_sales_yoy,q_profit_yoy
74,000009.SZ,2015-04-29,21.07,3.354570e+06,电池化学品,2015-04-29,2015-03-31,15.1639,-13.6133
75,000009.SZ,2015-04-30,22.38,3.563136e+06,电池化学品,2015-04-29,2015-03-31,15.1639,-13.6133
76,000009.SZ,2015-05-04,22.37,3.561544e+06,电池化学品,2015-04-29,2015-03-31,15.1639,-13.6133
77,000009.SZ,2015-05-05,21.80,3.470794e+06,电池化学品,2015-04-29,2015-03-31,15.1639,-13.6133
78,000009.SZ,2015-05-06,22.10,3.518557e+06,电池化学品,2015-04-29,2015-03-31,15.1639,-13.6133
...,...,...,...,...,...,...,...,...,...
748672,920682.BJ,2025-10-13,9.82,3.186394e+05,线缆部件及其他,2025-08-23,2025-06-30,-0.5330,-1.5003
748673,920682.BJ,2025-10-14,10.00,3.244800e+05,线缆部件及其他,2025-08-23,2025-06-30,-0.5330,-1.5003
748674,920682.BJ,2025-10-15,10.06,3.264269e+05,线缆部件及其他,2025-08-23,2025-06-30,-0.5330,-1.5003
748675,920682.BJ,2025-10-16,9.91,3.215597e+05,线缆部件及其他,2025-08-23,2025-06-30,-0.5330,-1.5003


In [59]:
print(f"--- 准备对 {len(df)} 行数据进行参数扫描 ---")

# --- 1. 定义参数网格 ---
param_grid = {
    "Y_PROFIT_GROWTH": [20.0, 30.0, 50.0,60,70,80,90,100],
    "Y_SALES_GROWTH": [10.0, 20.0,30,40,50,60,70,80,90,100],
    "Z_QUARTERS": [2],
    "X_INDUSTRY_PCT": [0.4, 0.5, 0.6, 0.7,0.8,0.9],
    "M_MONTHS": [1, 3, 6],
    # 【修改】增加 'PROFIT_ONLY' 和 'SALES_ONLY'
    "Y_GROWTH_LOGIC": ['AND', 'OR', 'PROFIT_ONLY', 'SALES_ONLY'] 
}

all_results = [] # 存储所有运行结果

# --- 2. 【优化】在循环外计算一次未来收益 ---
# 这一步在所有循环中都是一样的，没必要重复计算
print("--- 正在预先计算所有未来收益... ---")
df_with_returns = df.copy()
df_with_returns = df_with_returns.sort_values(by=['ts_code', 'trade_date'])
for m in tqdm(param_grid["M_MONTHS"], desc="Calculating Future Returns"):
    m_days = m * 21
    return_col = f'future_return_{m}M'
    future_price = df_with_returns.groupby('ts_code')['close'].shift(-m_days)
    df_with_returns[return_col] = (future_price / df_with_returns['close']) - 1
print("--- 未来收益计算完毕 ---")


# --- 3. 开始参数循环 ---
# 使用 product 来自动生成所有组合
from itertools import product

param_combinations = list(product(
    param_grid["Y_PROFIT_GROWTH"],
    param_grid["Y_SALES_GROWTH"],  # <--- Y_profit 和 Y_sales 来自这里
    param_grid["Z_QUARTERS"],
    param_grid["X_INDUSTRY_PCT"],
    param_grid["Y_GROWTH_LOGIC"]
))

# 循环主体
for Y_profit, Y_sales, Z, X, GROWTH_LOGIC in tqdm(param_combinations, desc="Scanning Parameters"):
    
    # --- 步骤 1: 提取唯一的季度财报，并计算“个股信号” ---
    financial_cols = ['ts_code', 'l3_name', 'ann_date', 'end_date', 'q_sales_yoy', 'q_profit_yoy']
    quarterly_df = df[financial_cols].drop_duplicates(subset=['ts_code', 'end_date']).copy()
    quarterly_df = quarterly_df.sort_values(by=['ts_code', 'end_date'])

    # 1.3/1.4: 【修改】标记并组合，以支持 'PROFIT_ONLY' 和 'SALES_ONLY'
    logic_upper = GROWTH_LOGIC.upper()

    if logic_upper == 'PROFIT_ONLY':
        # 只考虑利润
        quarterly_df['is_growth'] = quarterly_df['q_profit_yoy'] > Y_profit
        
    elif logic_upper == 'SALES_ONLY':
        # 只考虑营收
        quarterly_df['is_growth'] = quarterly_df['q_sales_yoy'] > Y_sales
        
    elif logic_upper == 'AND':
        # 必须两者都满足
        is_growth_profit = quarterly_df['q_profit_yoy'] > Y_profit
        is_growth_sales = quarterly_df['q_sales_yoy'] > Y_sales
        quarterly_df['is_growth'] = is_growth_profit & is_growth_sales
        
    elif logic_upper == 'OR':
        # 满足任意一个即可
        is_growth_profit = quarterly_df['q_profit_yoy'] > Y_profit
        is_growth_sales = quarterly_df['q_sales_yoy'] > Y_sales
        quarterly_df['is_growth'] = is_growth_profit | is_growth_sales
        
    else:
        print(f"Warning: Logic {GROWTH_LOGIC} 不支持. Skipping.")
        quarterly_df['is_growth'] = False # 默认不满足

    # 1.5 计算连续Z季 (这部分不变)
    rolling_sum = quarterly_df.groupby('ts_code')['is_growth'].rolling(window=Z).sum()
    quarterly_df['consecutive_growth_count'] = rolling_sum.reset_index(level=0, drop=True)

    # 1.6 生成个股信号 (这部分不变)
    quarterly_df['stock_signal'] = (quarterly_df['consecutive_growth_count'] == Z)
    # --- 步骤 2: 聚合个股信号，计算“行业信号” ---
    industry_pct = quarterly_df.groupby(['ann_date', 'l3_name'])['stock_signal'].mean()
    industry_signal_df = industry_pct.reset_index(name='industry_pct')
    industry_signal_df['strategy_signal'] = (industry_signal_df['industry_pct'] > X)
    good_industries = industry_signal_df[industry_signal_df['strategy_signal'] == True][['ann_date', 'l3_name']].drop_duplicates()

    # --- 步骤 3: (已在循环外完成) ---
    
    # --- 步骤 4: 执行回测 (对每个 M) ---
    good_stocks = quarterly_df[quarterly_df['stock_signal'] == True][['ts_code', 'l3_name', 'ann_date']]
    trade_list = pd.merge(good_stocks, good_industries, on=['ann_date', 'l3_name'], how='inner')
    trade_list = trade_list[['ts_code', 'l3_name', 'ann_date']].drop_duplicates()
    
    if trade_list.empty:
        # 如果这个 (Y, Z, X, Logic) 组合没产生任何信号，就为所有的 M 记录空结果
        for M in param_grid["M_MONTHS"]:
            all_results.append({
                "Y": Y, "Z": Z, "X": X, "M": M, "Logic": GROWTH_LOGIC,
                "avg_return": np.nan, "median_return": np.nan, "win_rate": np.nan,
                "std_dev": np.nan, "sharpe_ratio": np.nan, "num_signals": 0
            })
        continue # 跳过这组参数的 M 循环

    # 准备价格数据 (只需一次)
    trade_list_sorted = trade_list.sort_values(by='ann_date')
    
    for M in param_grid["M_MONTHS"]:
        return_col = f'future_return_{M}M'
        
        # 准备价格数据
        price_df_processed = df_with_returns[['ts_code', 'trade_date', 'l3_name', return_col]].copy()
        price_df_sorted = price_df_processed.sort_values(by='trade_date')

        # 4.5 使用 merge_asof 匹配
        results_df = pd.merge_asof(
            trade_list_sorted,
            price_df_sorted,
            left_on='ann_date',
            right_on='trade_date',
            by='ts_code',
            direction='forward',
            suffixes=('_signal', '_price')
        )
        results_df.dropna(subset=['trade_date', return_col], inplace=True)

        # --- 步骤 5: 分析结果 ---
        if results_df.empty:
            avg_return, median_return, win_rate, std_dev, sharpe_ratio, num_signals = np.nan, np.nan, np.nan, np.nan, np.nan, 0
        else:
            portfolio_returns = results_df.groupby(['trade_date', 'l3_name_signal'])[return_col].mean()
            
            avg_return = portfolio_returns.mean()
            median_return = portfolio_returns.median()
            win_rate = (portfolio_returns > 0).mean()
            num_signals = len(portfolio_returns)
            std_dev = portfolio_returns.std()
            
            annualization_factor = np.sqrt(12 / M) if M > 0 else 1
            sharpe_ratio = (avg_return / std_dev) * annualization_factor if std_dev > 0 else np.nan

        # 记录结果
        all_results.append({
            "Y": Y, "Z": Z, "X": X, "M": M, "Logic": GROWTH_LOGIC,
            "avg_return": avg_return,
            "median_return": median_return,
            "win_rate": win_rate,
            "std_dev": std_dev,
            "sharpe_ratio": sharpe_ratio,
            "num_signals": num_signals
        })
        
        if num_signals > 0 and not pd.isna(avg_return) and avg_return > 0.05:
            
            # 1. 定义文件夹和文件名
            RESULTS_DIR = "results" # (图表和Excel保存在同一处)
            os.makedirs(RESULTS_DIR, exist_ok=True) # 确保文件夹存在
            
            # 使用所有参数创建唯一的标题和文件名
            plot_title = (
                f"Strategy (Yp={Y_profit}, Ys={Y_sales}, Z={Z}, X={X}, M={M}, Logic={GROWTH_LOGIC})\n"
                f"Sharpe: {sharpe_ratio:.2f} | Avg Return: {avg_return:.2%} | Win Rate: {win_rate:.2%}"
            )
            plot_filename = os.path.join(
                RESULTS_DIR, 
                f"plot_Yp{Y_profit}_Ys{Y_sales}_Z{Z}_X{X}_M{M}_Logic{GROWTH_LOGIC}.png"
            )

            # 2. 尝试绘图和保存
            try:
                plt.figure(figsize=(10, 6))
                portfolio_returns.hist(bins=30, alpha=0.75, edgecolor='black')
                plt.axvline(avg_return, color='red', linestyle='--', linewidth=2, label=f'平均收益 ({avg_return:.2%})')
                plt.axvline(median_return, color='orange', linestyle=':', linewidth=2, label=f'中位收益 ({median_return:.2%})')
                plt.title(plot_title)
                plt.xlabel("收益率")
                plt.ylabel("次数")
                plt.legend()
                plt.grid(False)
                
                # 保存图表，而不是 plt.show()
                plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
                
            except Exception as e:
                print(f"!! 保存图表失败: {plot_filename}. 错误: {e}")
            
            # 3. 关闭图表，释放内存 (非常重要!)
            plt.close()


# --- 6. 汇总所有结果 ---
print("\n--- 自动化参数扫描完成 ---")

results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values(by="avg_return", ascending=False)
results_df = results_df.round(3)

print("参数扫描结果汇总 (按平均收益率排序):")
display(results_df) # 在 ipynb 中， display() 比 print() 更漂亮

--- 准备对 720058 行数据进行参数扫描 ---
--- 正在预先计算所有未来收益... ---


Calculating Future Returns:   0%|          | 0/3 [00:00<?, ?it/s]

--- 未来收益计算完毕 ---


Scanning Parameters:   0%|          | 0/1920 [00:00<?, ?it/s]


--- 自动化参数扫描完成 ---
参数扫描结果汇总 (按平均收益率排序):


,Y,Z,X,M,Logic,avg_return,median_return,win_rate,std_dev,sharpe_ratio,num_signals
2156,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1796,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1868,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1940,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1652,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
...,...,...,...,...,...,...,...,...,...,...,...
5666,50.0,2,0.8,6,AND,-0.130,-0.212,0.231,0.358,-0.512,91
5678,50.0,2,0.9,6,AND,-0.130,-0.212,0.231,0.358,-0.512,91
2102,50.0,2,0.5,6,AND,-0.131,-0.185,0.233,0.344,-0.539,103
5642,50.0,2,0.6,6,AND,-0.135,-0.210,0.219,0.353,-0.540,96


In [60]:
results_df.describe()

,Y,Z,X,M,avg_return,median_return,win_rate,std_dev,sharpe_ratio,num_signals
count,5760.0,5760.0,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000,5760.000000
mean,50.0,2.0,0.650000,3.333333,0.013966,-0.051171,0.409409,0.339911,0.037154,590.649132
std,0.0,0.0,0.170797,2.054983,0.041638,0.046163,0.059124,0.147322,0.241320,430.278348
min,50.0,2.0,0.400000,1.000000,-0.140000,-0.214000,0.214000,0.153000,-0.800000,82.000000
25%,50.0,2.0,0.500000,1.000000,-0.012000,-0.061000,0.384000,0.171000,-0.080000,248.000000
50%,50.0,2.0,0.650000,3.000000,0.011000,-0.034000,0.427000,0.341000,0.121000,490.000000
75%,50.0,2.0,0.800000,6.000000,0.050000,-0.017000,0.454000,0.483000,0.192000,834.000000
max,50.0,2.0,0.900000,6.000000,0.084000,-0.004000,0.489000,0.660000,0.372000,2680.000000


In [65]:
results_df[results_df['avg_return']>0.08]

,Y,Z,X,M,Logic,avg_return,median_return,win_rate,std_dev,sharpe_ratio,num_signals
2156,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1796,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1868,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1940,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1652,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
2012,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1508,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1580,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
1724,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
2084,50.0,2,0.9,6,PROFIT_ONLY,0.084,-0.048,0.437,0.562,0.212,600
